Updated

In [6]:
import pandas as pd
import time
import re
import logging
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Function to read company names and job titles from Excel
def read_company_names_and_titles(file_path):
    df = pd.read_excel(file_path)
    return df[['Company Name', 'Job Title']].to_dict('records'), df

# Function to initialize the Chrome driver
def initialize_driver(path):
    service = Service(path)
    driver = webdriver.Chrome(service=service)
    return driver

# Function to search for company contact pages and extract contact details
def company_details(driver, company_name, job_title):
    url = "https://www.google.com/"
    driver.get(url)
    time.sleep(3)

    contact_info = {}

    try:
        search_box = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//textarea[@name='q']")))
        search_box.clear()
        search_box.send_keys(f"{company_name} contact")
        search_box.submit()
        
        first_result = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "h3")))
        first_result.click()

        page_source = driver.page_source
        soup = BeautifulSoup(page_source, "html.parser")

        phone_pattern = re.compile(r'\b(?:\+1[-.\s]?)?\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}\b')
        email_pattern = re.compile(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}')

        phones = phone_pattern.findall(soup.get_text())
        emails = email_pattern.findall(soup.get_text())

        contact_info['phones'] = phones if phones else ['N/A']
        contact_info['emails'] = emails if emails else ['N/A']
        contact_info['company'] = company_name
        contact_info['job_title'] = job_title
        
    except Exception as e:
        logging.error(f"Failed to retrieve contact details for {company_name}: {e}")
        contact_info = {
            'company': company_name,
            'job_title': job_title,
            'phones': ['N/A'],
            'emails': ['N/A']
        }

    return contact_info

# Function to save or update contact details in the Excel file
def save_to_excel(file_path, original_df, contact_details):
    # Create a DataFrame from the contact details
    contact_df = pd.DataFrame(contact_details)
    
    # Merge the original DataFrame with the contact details DataFrame
    merged_df = pd.merge(original_df, contact_df, left_on=['Company Name', 'Job Title'], right_on=['company', 'job_title'], how='left')
    
    # Drop duplicate columns
    merged_df.drop(columns=['company', 'job_title'], inplace=True)
    
    # Save the merged DataFrame to Excel
    merged_df.to_excel(file_path, index=False)

# Main execution
def main():
    file_path = 'jobs_us_Demo.ods'
    output_file_path = 'company_contact_details_Dice_US_Part8_2.xlsx'
    chrome_driver_path = "/home/saif/vscode/chromedriver"
    
    company_data_list, original_df = read_company_names_and_titles(file_path)
    driver = initialize_driver(chrome_driver_path)

    all_contact_details = []
    processed_companies = {}
    counter = 0
    total_companies = len(company_data_list)

    for company_data in company_data_list:
        company_name = company_data['Company Name']
        job_title = company_data['Job Title']
        
        if company_name in processed_companies:
            logging.info(f"Using cached details for company: {company_name}")
            contact_details = processed_companies[company_name]
            contact_details['job_title'] = job_title  # Update job title
        else:
            contact_details = company_details(driver, company_name, job_title)
            processed_companies[company_name] = contact_details
        
        all_contact_details.append(contact_details)
        
        counter += 1
        logging.info(f"Processed {counter}/{total_companies}: {contact_details}")

        if counter == total_companies:
            break

    driver.quit()

    # Save the contact details to the Excel file
    save_to_excel(output_file_path, original_df, all_contact_details)

    logging.info("Processing complete. All companies have been processed.")

if __name__ == "__main__":
    main()

2024-07-04 10:04:32,513 - INFO - Processed 1/24: {'phones': ['N/A'], 'emails': ['employeeservices@apexsystems.com'], 'company': 'Apex Systems', 'job_title': 'Data Engineer'}
2024-07-04 10:07:50,539 - INFO - Processed 2/24: {'phones': ['800) 447-0663', '800) 447-0663', '201) 735-2622', '203) 356-7000', '305) 884-8200', '407) 363-8000', '818) 840-4444', '818) 777-1000', '203 618 8000', '866) 787-6228', '212) 664-4444', '8004470663', '(212) 664-3914', '8004470663'], 'emails': ['nbcucareers@nbcuni.com'], 'company': 'NBC Universal', 'job_title': 'Sr. Staff Software Engineer (Developer Experience)'}
2024-07-04 10:07:58,766 - INFO - Processed 3/24: {'phones': ['N/A'], 'emails': ['N/A'], 'company': 'Microsoft Corporation', 'job_title': 'Principal Software Engineer'}
2024-07-04 10:07:58,767 - INFO - Using cached details for company: Apex Systems
2024-07-04 10:07:58,767 - INFO - Processed 4/24: {'phones': ['N/A'], 'emails': ['employeeservices@apexsystems.com'], 'company': 'Apex Systems', 'job_ti